In [1]:
from src.hurdle import Hurdle
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
model = Hurdle()

In [8]:
df_policy = pd.read_csv('data/PolicyLevel.csv',
parse_dates=['EnrollDate', 'CancelDate'],
infer_datetime_format=True)
df_policy.head(2)

,PolicyId,EnrollDate,CancelDate,MonthlyPremium
0,92597,2010-12-07,NaT,34.54
1,92808,2010-11-09,NaT,18.54


In [9]:

df_claim = pd.read_csv('data/ClaimLevel.csv',
parse_dates=['ClaimDate'],
infer_datetime_format=True)
df_claim['claims'] = 1
df_claim.head(2)

,PolicyId,ClaimDate,ClaimedAmount,PaidAmount,claims
0,148971,2016-02-11,924.39,814.27,1
1,566332,2016-01-20,155.43,0.00,1


In [14]:
df_claim = df_claim.groupby('PolicyId').sum().reset_index()
df_claim.head()

,PolicyId,ClaimedAmount,PaidAmount,claims
0,10,5677.16,4751.76,10
1,15,505.24,385.69,9
2,76,2440.86,1238.74,14
3,97,187.02,0.00,1
4,133,287.06,79.70,11


In [16]:
df = pd.merge(df_policy, df_claim, how='right', on='PolicyId', copy=True, indicator=False)
df.pop('CancelDate')
df.pop('EnrollDate')
df.pop('PaidAmount')
df.pop('PolicyId')
df.head()

,MonthlyPremium,ClaimedAmount,claims
0,46.00,5677.16,10
1,23.83,505.24,9
2,40.06,2440.86,14
3,16.75,187.02,1
4,19.30,287.06,11


In [17]:
y = np.array(df.pop('ClaimedAmount')).flatten()
X = df.values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

In [18]:
model.fit(Xtrain, ytrain)
pred = model.predict(Xtest)

In [19]:
ytest[pred==0]

array([], dtype=float64)

In [20]:
model.pred_x_class_[model.pred_x_class_==0]

array([], dtype=int32)

In [21]:
ev = model.predict_expected_value(Xtrain)

In [22]:
ev

array([ 539.21271641,  406.77455673,  841.02702101, ...,  519.77560937,
       1085.84190776,  449.78755168])

In [23]:
pred

array([ 666.71311647,  837.9328773 ,  495.62542785, ...,  391.03009621,
       2782.82519525,  332.65750843])